In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Linkhon_Hasan_CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################

rescue_queries = {
    'Water': {
        'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
        'sex_upon_outcome': {'$in': ['Intact Female']},
        'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
    },
    'Mountain': {
        'breed': {'$in': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
        'sex_upon_outcome': {'$in': ['Intact Male']},
        'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
    },
    'Disaster': {
        'breed': {'$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
        'sex_upon_outcome':{'$in': ['Intact Male']},
        'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
    }
}
username = "aacuser"
password = "mongopass"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.getAnimal({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Logo.png'  
try:
    with open(image_filename, 'rb') as img:
        encoded_image = base64.b64encode(img.read()).decode('utf-8')
except FileNotFoundError:
    print(f"Error: {image_filename} not found.")
except Exception as e:
    print(f"Could not load image: {e}")


app.layout = html.Div([
    html.Center(html.B(html.H1('Final Project Dashboard'))),
    html.Div(html.A(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px'}), href='http://www.snhu.edu')),
    html.Div("Linkhon Hasan", style={'font-size': '20px', 'margin-bottom': '20px'}),
    html.Hr(),
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        clearable=False,
        style={'width': '50%'}
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                        row_selectable='single', 
                        page_size=10,  
                        sort_action='native', 
                        selected_rows=[0],  

                        ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    print(f"Filter type selected: {filter_type}")

    if filter_type == 'Reset':
        return df.to_dict('records')

    query = rescue_queries.get(filter_type)

    if query:
        filtered_data = db.getAnimal(query)
        filtered_df = pd.DataFrame.from_records(filtered_data)
        if '_id' in filtered_df.columns:
            filtered_df.drop(columns=['_id'], inplace=True)
        print(f"Filtered data shape: {filtered_df.shape}")
        
        if filtered_df.empty:
            print("No results found. Returning all data.")
            return df.to_dict('records')
        
        return filtered_df.to_dict('records')

    return df.to_dict('records')

#Pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    figure = px.pie(dff, names='breed', title='Preferred Animal Breeds')
    
    figure.update_layout(
        autosize=False,
        width=800,
        height=800,
        margin=dict(l=20, r=20, t=40, b=20),
    )
    
    figure.update_traces(
        hole=0.3, 
        textposition='inside',
        textinfo='percent+label',
    )

    return [dcc.Graph(figure=figure)]

# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None or index is None or len(index) == 0:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty:
        return
    
    row = index[0]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=False)


 * Running on http://127.0.0.1:10597/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2024 03:35:46] "GET /_alive_56b9b39e-1eaa-4a8c-9586-657788d30a04 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:10597/


127.0.0.1 - - [20/Oct/2024 03:44:18] "POST /_dash-update-component HTTP/1.1" 200 -


Filter type selected: Reset


127.0.0.1 - - [20/Oct/2024 03:44:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:44:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:46:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:46:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:46:12] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [20/Oct/2024 03:53:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 03:53:50] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 04:22:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 04:22:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 04:22:22] "POST /_dash-update-component HTTP/1.1" 200 -


Filter type selected: Disaster
Filtered data shape: (4, 16)


127.0.0.1 - - [20/Oct/2024 04:22:24] "POST /_dash-update-component HTTP/1.1" 200 -


Filter type selected: Reset


127.0.0.1 - - [20/Oct/2024 04:22:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2024 04:22:25] "POST /_dash-update-component HTTP/1.1" 200 -
